<span style="font-weight:bold; font-size:20px;">Import</span>

In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import copy
from datetime import datetime
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import PolynomialFeatures

<span style="font-weight:bold; font-size:20px;">All function</span>

In [ ]:
# Data preprocessing
def data_pre(companies):
    for i in companies:
        if 'metadata' in i.keys():
            del i['metadata']
        comp_name = i['name']
        i['name'] = f"{comp_name}_{companies.index(i)}"
        if not isinstance(i['equity_funding_total_usd'], int):
            value = i['equity_funding_total_usd']
            i['equity_funding_total_usd'] = int(value['$numberLong'])
        if i['num_funding_rounds'] is None:
            i['num_funding_rounds'] = 0
    return companies

In [ ]:
# The criteria to judge whether the company is successful
def whether_success(companies):
    for i in companies:
        i['startup'] = 'failed'
        if i['operating_status'] != 'closed':
            if i['went_public_on']:
                i['startup'] = 'success'
            elif i['status'] == 'was_acquired' or i['status'] == 'ipo':
                i['startup'] = 'success'
            elif (datetime.now() - datetime.strptime(i['founded_on'], '%Y-%m-%d')).days/365 >= 7:
                i['startup'] = 'success'
            elif i['num_funding_rounds'] >= 7:
                i['startup'] = 'success'
            elif i['equity_funding_total_usd'] >= 100000000:
                i['startup'] = 'success'
    return companies

In [ ]:
# Network building
def build_nw(comps, peo):
    G = nx.Graph()
    G.add_nodes_from(comps)
    
    peo_dict = {comp: set(p) for comp, p in zip(comps, peo)}
    
    def process_pairs(i, comp):
        edges = []
        for j in range(i+1, len(comps)):
            same = peo_dict[comp] & peo_dict[comps[j]]
            if same:
                edges.append((comp, comps[j], len(same)))
        return edges

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_pairs, i, comp) for i, comp in enumerate(comps)]
        for future in as_completed(futures):
            for comp1, comp2, weight in future.result():
                if G.has_edge(comp1, comp2):
                    G[comp1][comp2]['weight'] += weight
                else:
                    G.add_edge(comp1, comp2, weight=weight)

    return G

In [ ]:
# Calculate success rate
def success_rate(data, dict, field):
    result = {}
    for i in dict:
        result[i] = 0
        for n in data:
            if isinstance(n[field], list):
                if i in n[field] and n['startup'] == 'success':
                    result[i] += 1
            else:
                if n[field] == i and n['startup'] == 'success':
                    result[i] += 1
    for k in result:
        result[k] = result[k]/dict[k]
    return result

In [ ]:
# Calculate component data
def components_data(G):
    result = {}
    for component in nx.connected_components(G):
        subgraph = G.subgraph(component)
        sub_size = len(subgraph)
        sub_success = 0
        for node in subgraph:
            if G.nodes[node]['success'] == 'success':
                sub_success += 1
        sub_success_rate = sub_success/sub_size
        for node in subgraph:
            result[node] = {'component_size': sub_size, 'component_success_rate': sub_success_rate}
    return result

In [ ]:
# Basic analysis of the whole network
def generate_data(G):
    net_nodes = len(G.nodes())
    net_edges = len(G.edges())
    net_den = round(nx.density(G),4)
    avg_cluster = round(nx.average_clustering(G),4)
    avg_degree = round(np.mean(list(dict(G.degree()).values())),4)
    max_degree = max(list(dict(G.degree()).values()))
    min_degree = min(list(dict(G.degree()).values()))
    total_strength = 0
    for node in G.nodes():
        node_strength = sum(data["weight"] for u, v, data in G.edges(node, data=True))
        total_strength += node_strength
    avg_strength = round(total_strength / net_nodes, 4)
    try:
        net_assortativity = round(nx.degree_assortativity_coefficient(G),4)
    except RuntimeWarning:
        net_assortativity = None

    table ={
        "Number of Nodes": net_nodes,
        "Number of Edges": net_edges,
        "Density": net_den,
        "Avg Clustering Coefficient": avg_cluster,
        "Avg Degree": avg_degree,
        "Max Degree": max_degree,
        "Min Degree": min_degree,
        "Avg Strength": avg_strength,
        "Assortativity": net_assortativity}    
    return table

In [ ]:
# Generate the features
def generate_feature(G, data):
    degrees = dict(G.degree())
    # clustering = nx.clustering(G)
    closeness = nx.closeness_centrality(G)
    # betweenness = nx.betweenness_centrality(G)
    # eigenvector_centrality = nx.eigenvector_centrality(G)
    # pagerank = nx.pagerank(G)
    
    nodes = [node for node in G.nodes() if node not in list(nx.isolates(G))]
    fea_info = {
        'node_id': nodes,
        'degree': [degrees[node] for node in nodes],
        # 'clustering': [clustering[node] for node in nodes],
        'closeness': [closeness[node] for node in nodes],
        # 'betweenness': [betweenness[node] for node in nodes],
        # 'closeness': [closeness[node] for node in nodes],
        # 'eigenvector_centrality': [eigenvector_centrality[node] for node in nodes],
        # 'pagerank': [pagerank[node] for node in nodes],
        'categories': [comp['category_groups'] for comp in data if comp['name'] not in list(nx.isolates(G))]
    }
    df = pd.DataFrame(fea_info)
    return df

In [ ]:
# Show the result
def evaluate_pred(model, X_test, y_test, y_pred, model_name):
    y_proba = model.predict_proba(X_test)[:, 1]
    print(f"{model_name} Classification Report:")
    print(classification_report(y_test, y_pred))
    
    if y_proba is not None:
        roc_auc = roc_auc_score(y_test, y_proba)
        print(f"{model_name} ROC AUC Score: {roc_auc:.3f}")
    else:
        print(f"{model_name} does not provide probability predictions for ROC AUC Score.")

<span style="font-weight:bold; font-size:20px;">Data loading and preprocessing</span>

In [ ]:
file = '/Users/kiening/Documents/Dissertation/dissertation.cb_companies.json'

with open(file, 'r', encoding='utf-8') as file:
    data = json.load(file)


In [ ]:
data = whether_success(data_pre(data))

<span style="font-weight:bold; font-size:20px;">Network building</span>

In [ ]:
comps = []
peo = []

for com in data:
    comps.append(com['name'])
    peo.append(list(set(com['founders']) | set(com['investors'])))

unique_comps = [f"{comp}_{i}" for i, comp in enumerate(comps)]

In [ ]:
G = build_nw(unique_comps, peo)

In [ ]:
# Draw network here
pos = nx.spring_layout(G)
nx.draw(G, 
        pos, 
        with_labels=False, 
        node_size=5, 
        font_size=5)
labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
plt.show()

In [ ]:
node_attri = {}
for index, node in enumerate(G.nodes()):
    node_attri[node] = {}
    node_attri[node]['categories'] = data[index]['category_groups']
    node_attri[node]['success'] = data[index]['startup']
nx.set_node_attributes(G, node_attri)

<span style="font-weight:bold; font-size:20px;">Basic analysis of the whole network</span>

In [ ]:
analysis = generate_data(G)
analysis

<span style="font-weight:bold; font-size:20px;">Isolated nodes</span>

In [ ]:
isolates = list(nx.isolates(G))
iso_comps = []
found_on = []
funding = []
rounds = []
startup = []
for i in range(len(unique_comps)):
    if unique_comps[i] in isolates:
        iso_comps.append(data_test[i]['name'])
        found_on.append((datetime.now() - datetime.strptime(data_test[i]['founded_on'], '%Y-%m-%d')).days)
        funding.append(data_test[i]['equity_funding_total_usd'])
        rounds.append(data_test[i]['num_funding_rounds'])
        startup.append(data_test[i]['startup'])
iso_df = pd.DataFrame({
    'company': iso_comps,
    'days_since_founded': found_on,
    'equity_funding_total_usd': funding,
    'num_funding_rounds': rounds,
    'startup': startup
})
iso_df['num_funding_rounds'] = iso_df['num_funding_rounds'].fillna(0)

In [ ]:
# Use Random forests to judge feature importance
X = iso_df.drop(columns=['startup', 'company'])
y = iso_df['startup']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalized
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Feature importance
feature_importances = model.feature_importances_
features = X.columns
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})

# Rank
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)

In [ ]:
# Feature combination
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

model_poly = RandomForestClassifier(random_state=42)
model_poly.fit(X_train_poly, y_train)

# Performance
y_pred = model_poly.predict(X_test_poly)
print("Classification Report with Interaction Features:")
print(classification_report(y_test, y_pred))

<span style="font-weight:bold; font-size:20px;">Non-isolated nodes</span>

In [ ]:
# Generate the feature dataframe
data_non_iso = [i for i in data if i['name'] not in isolates]
fea_df = generate_feature(G, data)
fea_df

In [ ]:
# Caculate the class success rate and closeness
cla = {}
for i in data_non_iso:
    for n in i['category_groups']:
        if not n in cla.keys():
            cla[n] = 1
        else:
            cla[n] += 1
cla_success_rate = success_rate(data_non_iso, cla, 'category_groups')
cla_avg_closeness = {}
for i in cla.keys():
    for n in range(len(fea_df['node_id'])):
        if i in fea_df['categories'][n]:
            if i in cla_avg_closeness:
                cla_avg_closeness[i] += fea_df['closeness'][n]
            else:
                cla_avg_closeness[i] = fea_df['closeness'][n]
for i in cla_avg_closeness:
    cla_avg_closeness[i] = cla_avg_closeness[i]/cla[i]


avg_cla_sco = []
avg_cla_clo = []
for i in fea_df['categories']:
    cla_rates = [cla_success_rate[n] for n in i]
    clo_rates = [cla_avg_closeness[n] for n in i]
    if cla_rates:
        avg_success_rate = sum(cla_rates) / len(cla_rates)
        avg_clo = sum(clo_rates) / len(clo_rates)
        avg_cla_sco.append(avg_success_rate)
        avg_cla_clo.append(avg_clo)
    else:
        avg_cla_sco.append(0)
        avg_cla_clo.append(0)
fea_df['class_score'] = avg_cla_sco
fea_df['class_avg_closeness'] = avg_cla_clo

In [ ]:
# Caculate the location success rate
location = {}
for i in data_non_iso:
    if i['location_groups']:
        for n in i['location_groups']:
            if not n in location:
                location[n] = 1
            else:
                location[n] += 1
loc_success_rate = success_rate(data_non_iso, location, 'location_groups')
avg_loc_sco = []
for i in data_non_iso:
    rates = [loc_success_rate[n] for n in i['location_groups']]
    if rates:
        avg_success_rate = sum(rates) / len(rates)
        avg_loc_sco.append(avg_success_rate)
    else:
        avg_loc_sco.append(0)
fea_df['location_score'] = avg_loc_sco

In [ ]:
# Caculate the number of employees success rate
employees = {}
for i in data_non_iso:
    if i['num_employees_enum']:
        if not i['num_employees_enum'] in employees:
            employees[i['num_employees_enum']] = 1
        else:
            employees[i['num_employees_enum']] += 1
emp_success_rate = success_rate(data_non_iso, employees, 'num_employees_enum')
avg_emp_sco = []
for i in data_non_iso:
    if i['num_employees_enum']:
        avg_emp_sco.append(emp_success_rate[i['num_employees_enum']])
    else:
        avg_emp_sco.append(0)
fea_df['employees_score'] = avg_emp_sco

In [ ]:
# Caculate component data
node_component_data = components_data(G)
comp_success_rate = [node_component_data[node]['component_success_rate'] for node in fea_df['node_id']]
comp_size = [node_component_data[node]['component_size'] for node in fea_df['node_id']]
fea_df['component_success_rate'] = comp_success_rate
fea_df['component_size'] = comp_size

<span style="font-weight:bold; font-size:20px;">Training and prediction</span>

In [ ]:
# Split the data
X = fea_df.drop(columns=['node_id', 'categories', 'component_size'])
y = [i['startup'] for i in data_non_iso]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Normalize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Feature importance
feature_importances = model.feature_importances_
features = X.columns
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})

# Rank
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)

In [ ]:
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

model_1 = LogisticRegression(max_iter=1000)
model_2 = DecisionTreeClassifier()
model_3 = RandomForestClassifier()
model_4 = SVC(kernel='linear', probability=True)
model_5 = KNeighborsClassifier(n_neighbors=3)

model_1.fit(X_train, y_train)
model_2.fit(X_train, y_train)
model_3.fit(X_train, y_train)
model_4.fit(X_train, y_train)
model_5.fit(X_train, y_train)

y_pred_1 = model_1.predict(X_test)
y_pred_2 = model_2.predict(X_test)
y_pred_3 = model_3.predict(X_test)
y_pred_4 = model_4.predict(X_test)
y_pred_5 = model_5.predict(X_test)

model_1_poly = LogisticRegression()
model_2_poly = DecisionTreeClassifier()
model_3_poly = RandomForestClassifier()
model_4_poly = SVC(kernel='linear', probability=True)
model_5_poly = KNeighborsClassifier(n_neighbors=3)

model_1_poly.fit(X_train_poly, y_train)
model_2_poly.fit(X_train_poly, y_train)
model_3_poly.fit(X_train_poly, y_train)
model_4_poly.fit(X_train_poly, y_train)
model_5_poly.fit(X_train_poly, y_train)

y_pred_1_poly = model_1_poly.predict(X_test_poly)
y_pred_2_poly = model_2_poly.predict(X_test_poly)
y_pred_3_poly = model_3_poly.predict(X_test_poly)
y_pred_4_poly = model_4_poly.predict(X_test_poly)
y_pred_5_poly = model_5_poly.predict(X_test_poly)

In [ ]:
evaluate_pred(model_1, X_test, y_test, y_pred_1, 'Logistic Regression')
evaluate_pred(model_2, X_test, y_test, y_pred_2, 'Decision Tree')
evaluate_pred(model_3, X_test, y_test, y_pred_3, 'Random Forest')
evaluate_pred(model_4, X_test, y_test, y_pred_4, 'SVC')
evaluate_pred(model_5, X_test, y_test, y_pred_5, 'K-Nearest Neighbors')
print('*******************************************')
evaluate_pred(model_1_poly, X_test_poly, y_test, y_pred_1_poly, 'Logistic Regression with Interaction Features')
evaluate_pred(model_2_poly, X_test_poly, y_test, y_pred_2_poly, 'Decision Tree with Interaction Features')
evaluate_pred(model_3_poly, X_test_poly, y_test, y_pred_3_poly, 'Random Forest with Interaction Features')
evaluate_pred(model_4_poly, X_test_poly, y_test, y_pred_4_poly, 'SVC with Interaction Features')
evaluate_pred(model_5_poly, X_test_poly, y_test, y_pred_5_poly, 'K-Nearest Neighbors with Interaction Features')